In [ ]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [ ]:
from MainSrc.PythonVersionHandler import *
from paths import *
from Sparker.SparkLogProcesser.SparkLogOperator import *
from Sparker.SparkLogProcesser.SparkLogReader import *
from Sparker.SparkLogProcesser.SparkLogAnalyzer import *
from Sparker.SparkLogProcesser.SparkLogFileHandler import *
from Sparker.PySparkImports import *
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
from Sparker.Logic.UpdatedProductPreferrer import *
from Sparker.Logic import UpdatedProductPreferrer

In [ ]:
def getUpdatedLabeledPairsOnJupyter(jour):
    journeys = jour.sortBy(lambda x: x['timestamp']).groupBy(lambda x: x['_c']).collect()
    events = []

    def lookActions(action, journey):
        if action['module'] == 'cart' or action['module'] == 'payment':
            journey = modulize(journey)
            id = action['id']
            if keyIn('search', journey):
                for log in journey['search']:
                    if keyIn('ids', log):
                        if id in log['ids']:
                            #events.append({'product': action['id'], 'action': action['module'], 'others': log['ids']})
                            events.append((action['id'], action['module'], log['ids']))

    for c, journey in journeys:
        for i in range(4, len(journeys)):
            lookActions(journey[i], journey[i-4:i])
            
    positive = 1
    negative = -1
    posPairs = []
    negPairs = []
    counts = {}
    labeledPairs = {}

    def posKey(id, module, otherId): return str((id, module, otherId))

    #idsSet = unique(list(map(lambda x: x['products'], events)))
    for id, module, others in events:
        for otherId in others:
            k = posKey(id, module, otherId)
            l = posKey(otherId, module, id)
            if keyIn(k, counts):
                counts[k] += 1
            else:
                counts[k] = 1
            posPairs.append((id, module, otherId, k))
            negPairs.append((otherId, module, id, l))

    for ke in list(counts.keys()):
        id, module, otherId = eval(ke)
        if module == 'payment':
            rev = posKey(otherId, module, id)
            if keyIn(rev, counts):
                if counts[ke] > counts[rev]:
                    labeledPairs[keyPairIds(id, otherId)] = positive
                    labeledPairs[keyPairIds(otherId, id)] = negative
        elif module == 'cart' and not keyIn(posKey(id, 'payment', otherId), counts) and not keyIn(posKey(otherId, 'payment', id), counts):
            rev = posKey(otherId, module, id)
            if keyIn(rev, counts):
                if counts[ke] > counts[rev]:
                    labeledPairs[keyPairIds(id, otherId)] = positive
                    labeledPairs[keyPairIds(otherId, id)] = negative
             
    labeledPairs = sc_().parallelize([(key, v) for key, v in labeledPairs.items()])
    print_(labeledPairs.count(), 'labeled pairs have been generated by', nowStr())
    return labeledPairs

In [ ]:
original = UpdatedProductPreferrer.getUpdatedLabeledPairs
UpdatedProductPreferrer.getUpdatedLabeledPairs = getUpdatedLabeledPairsOnJupyter

In [ ]:
from runGG_Project import main as run
run(trainLocalDataTest)